In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import ast
import json

In [2]:
results = pd.read_json('../scraping/data/constituency/Dum dum_2.json')
results = results.T
results

,Candidate,Party,Votes
0,SAUGATA ROY,All India Trinamool Congress,483244
1,ASIM KUMAR DASGUPTA,Communist Party of India (Marxist),328310
2,TAPAN SIKDAR,Bharatiya Janata Party,254819
3,DHANANJAY (SHAKTI) MOITRA,Indian National Congress,34116
4,ASHOK KUNDU,Party for Democratic Socialism,4823
5,NARENDRA NATH GHOSH (NAREN GHOSH),Bahujan Samaj Party,4077
6,SUKHA RANJAN BANIK,Independent,3256
7,SATYABRATA BANDYOPADHYAY,Rashtriya Janadhikar Suraksha Party,1618
8,SIKHA SEN ROY,Communist Party of India (Marxist-Leninist) Re...,1544
9,None of the Above,None of the Above,16837


In [3]:
def breakItUp(df, colName):
    v = json_normalize([ast.literal_eval(json.dumps(j)) for j in df[colName].dropna()], sep='_')
    v.index = df.index[df.pop(colName).notna()]
    df = df.join(v)
    return df

In [4]:
comparison = pd.read_json('../scraping/data/constituenciesComparison2014/19: DUM DUM\n_0.json')
comparison = comparison.T
comparison = breakItUp(comparison, 'Name')
comparison.rename(columns={'data': 'Name', 'url': 'ProfileLink'}, inplace=True)

comparison['Id'] = comparison['ProfileLink'].apply(lambda data: data.split("=")[-1])

comparison

,Age,Criminal Cases,Education Level,Gender,IPC Details*,Immovable Assets,Movable Assets,Number of Cases,PAN Given(Y or N),Party Code,Serious IPC Counts,Total Assets,Total Liabilities,Name,ProfileLink,Id
1,52,No,Doctorate,M,NaN,"65,00,000~ 65 Lacs+","13,36,368~ 13 Lacs+",0,Y,Party for Democratic Socialism,0,"78,36,368~ 78 Lacs+","5,15,000~ 5 Lacs+",Ashok Kundu,http://www.myneta.info/ls2014/http://myneta.in...,9794
2,68,No,Doctorate,M,NaN,"18,04,080~ 18 Lacs+","54,12,157~ 54 Lacs+",0,Y,CPI(M),0,"72,16,237~ 72 Lacs+",0~,Asim Kumar Dasgupta,http://www.myneta.info/ls2014/http://myneta.in...,9358
3,61,Yes,Graduate,M,"voluntarily causing grievous hurt(325), Assaul...","40,00,000~ 40 Lacs+","25,06,000~ 25 Lacs+",2,Y,INC,4,"65,06,000~ 65 Lacs+","1,50,000~ 1 Lacs+",Dhananjoy Moitra,http://www.myneta.info/ls2014/http://myneta.in...,9357
4,71,No,Graduate,M,NaN,"70,00,000~ 70 Lacs+","1,12,926~ 1 Lacs+",0,Y,BSP,0,"71,12,926~ 71 Lacs+",0~,Narendra Nath Ghosh,http://www.myneta.info/ls2014/http://myneta.in...,9359
5,52,No,Post Graduate,M,NaN,"13,00,000~ 13 Lacs+","22,74,024~ 22 Lacs+",0,Y,Rashtriya Janadhikar Suraksha Party,0,"35,74,024~ 35 Lacs+","10,000~ 10 Thou+",Satyabrata Bandyopadhyay,http://www.myneta.info/ls2014/http://myneta.in...,9795
6,66,No,Post Graduate,M,NaN,"34,50,000~ 34 Lacs+","1,60,02,010~ 1 Crore+",0,Y,AITC,0,"1,94,52,010~ 1 Crore+","25,00,000~ 25 Lacs+",Saugata Roy,http://www.myneta.info/ls2014/http://myneta.in...,9356
7,48,No,Graduate,F,NaN,"5,00,000~ 5 Lacs+","2,500~ 2 Thou+",0,N,CPI(ML) Red star,0,"5,02,500~ 5 Lacs+",0~,Sikha Sen Roy,http://www.myneta.info/ls2014/http://myneta.in...,9360
8,71,No,Graduate,M,NaN,"20,00,000~ 20 Lacs+","39,08,115~ 39 Lacs+",0,Y,BJP,0,"59,08,115~ 59 Lacs+",0~,Tapan Sikdar,http://www.myneta.info/ls2014/http://myneta.in...,9361


In [5]:
tableMapper = {
    "All India Trinamool Congress": "AITC",
    "Communist Party of India  (Marxist)": "CPI(M)",
    "Bharatiya Janata Party": "BJP",
    "Indian National Congress": "INC",
    "Party for Democratic Socialism": "Party for Democratic Socialism",
    "Bahujan Samaj Party": "BSP",
    "Independent": None,
    "Rashtriya Janadhikar Suraksha Party": "Rashtriya Janadhikar Suraksha Party",
    "Communist Party of India (Marxist-Leninist) Red Star": "CPI(ML) Red star",
    "None of the Above": None
}

In [6]:
for index, row in results.iterrows():
    foo = comparison.loc[comparison['Party Code'] == tableMapper[row['Party']]]
    
    if len(foo) == 0:
        foo = pd.DataFrame([["NaN"] * len(comparison.columns)], columns=comparison.columns)
        
    foo['Votes'] = row['Votes']
    foo['Name'] = row['Candidate']
    foo['Party Code'] = row['Party']

    try:
        completeTable = completeTable.append(foo)
    except:
        completeTable = foo
    
completeTable

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-co

,Age,Criminal Cases,Education Level,Gender,IPC Details*,Immovable Assets,Movable Assets,Number of Cases,PAN Given(Y or N),Party Code,Serious IPC Counts,Total Assets,Total Liabilities,Name,ProfileLink,Id,Votes
6,66,No,Post Graduate,M,NaN,"34,50,000~ 34 Lacs+","1,60,02,010~ 1 Crore+",0,Y,All India Trinamool Congress,0,"1,94,52,010~ 1 Crore+","25,00,000~ 25 Lacs+",SAUGATA ROY,http://www.myneta.info/ls2014/http://myneta.in...,9356,483244
2,68,No,Doctorate,M,NaN,"18,04,080~ 18 Lacs+","54,12,157~ 54 Lacs+",0,Y,Communist Party of India (Marxist),0,"72,16,237~ 72 Lacs+",0~,ASIM KUMAR DASGUPTA,http://www.myneta.info/ls2014/http://myneta.in...,9358,328310
8,71,No,Graduate,M,NaN,"20,00,000~ 20 Lacs+","39,08,115~ 39 Lacs+",0,Y,Bharatiya Janata Party,0,"59,08,115~ 59 Lacs+",0~,TAPAN SIKDAR,http://www.myneta.info/ls2014/http://myneta.in...,9361,254819
3,61,Yes,Graduate,M,"voluntarily causing grievous hurt(325), Assaul...","40,00,000~ 40 Lacs+","25,06,000~ 25 Lacs+",2,Y,Indian National Congress,4,"65,06,000~ 65 Lacs+","1,50,000~ 1 Lacs+",DHANANJAY (SHAKTI) MOITRA,http://www.myneta.info/ls2014/http://myneta.in...,9357,34116
1,52,No,Doctorate,M,NaN,"65,00,000~ 65 Lacs+","13,36,368~ 13 Lacs+",0,Y,Party for Democratic Socialism,0,"78,36,368~ 78 Lacs+","5,15,000~ 5 Lacs+",ASHOK KUNDU,http://www.myneta.info/ls2014/http://myneta.in...,9794,4823
4,71,No,Graduate,M,NaN,"70,00,000~ 70 Lacs+","1,12,926~ 1 Lacs+",0,Y,Bahujan Samaj Party,0,"71,12,926~ 71 Lacs+",0~,NARENDRA NATH GHOSH (NAREN GHOSH),http://www.myneta.info/ls2014/http://myneta.in...,9359,4077
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Independent,NaN,NaN,NaN,SUKHA RANJAN BANIK,NaN,NaN,3256
5,52,No,Post Graduate,M,NaN,"13,00,000~ 13 Lacs+","22,74,024~ 22 Lacs+",0,Y,Rashtriya Janadhikar Suraksha Party,0,"35,74,024~ 35 Lacs+","10,000~ 10 Thou+",SATYABRATA BANDYOPADHYAY,http://www.myneta.info/ls2014/http://myneta.in...,9795,1618
7,48,No,Graduate,F,NaN,"5,00,000~ 5 Lacs+","2,500~ 2 Thou+",0,N,Communist Party of India (Marxist-Leninist) Re...,0,"5,02,500~ 5 Lacs+",0~,SIKHA SEN ROY,http://www.myneta.info/ls2014/http://myneta.in...,9360,1544
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None of the Above,NaN,NaN,NaN,None of the Above,NaN,NaN,16837
